<a href="https://colab.research.google.com/github/eildes06/02_DS_DATA_SCIENCE/blob/main/recommendersystems1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Data/ml-1m'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/content/drive/MyDrive/Data/ml-1m/users.dat
/content/drive/MyDrive/Data/ml-1m/ratings.dat
/content/drive/MyDrive/Data/ml-1m/movies.dat
/content/drive/MyDrive/Data/ml-1m/README


In [ ]:
# pip install --upgrade tensorflow 

In [ ]:
!pip install LightFM

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 5.0 MB/s 
  Created wheel for LightFM: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705361 sha256=c8aef462c0066d252b3ad86f29cc8c78d729a074f92bdc2fe6bd071f3418bc71
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built LightFM


In [ ]:
from scipy import sparse
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

from lightfm.datasets import fetch_movielens

movielens = fetch_movielens()

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 4.5 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1634006 sha256=5267b7e55c3a882ecd57075fa2314c79c1575732262885fd09859c63d98fa979
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
from tabulate import tabulate
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly

import random
from pathlib import Path
import matplotlib.pyplot as plt
from collections import defaultdict

from surprise.model_selection import train_test_split

from surprise import KNNWithMeans  
from surprise import SVD 

from surprise.model_selection import GridSearchCV

from surprise import accuracy


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
column_names = ['User_ID', 'Gender', 'Age', 'Occupation', 'Zip-code']
users = pd.read_csv("/content/drive/MyDrive/Data/ml-1m/users.dat", sep = "::", names = column_names, engine='python')
users.head()
users.drop('Zip-code',axis = 1).head()

,User_ID,Gender,Age,Occupation
0,1,F,1,10
1,2,M,56,16
2,3,M,25,15
3,4,M,45,7
4,5,M,25,20


In [ ]:
users['Age']= users['Age'].apply(lambda x : x//10 +1 )
users.head()

,User_ID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,6,16,70072
2,3,M,3,15,55117
3,4,M,5,7,02460
4,5,M,3,20,55455


In [ ]:
column_names = ['User_ID', 'MovieID', 'Rating', 'Timestamp']
ratings = pd.read_csv("/content/drive/MyDrive/Data/ml-1m/ratings.dat", sep = "::", names = column_names, engine='python')
ratings.head()

,User_ID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
ratings_temp  = ratings.copy()

In [ ]:
ratings_temp.sort_values(by = 'Timestamp', inplace = True)

In [ ]:
column_names = ['MovieID', 'Title', 'Genres']
movies = pd.read_csv("/content/drive/MyDrive/Data/ml-1m/movies.dat", sep = "::", names = column_names, encoding='latin-1', engine='python')
movies.head() 

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
movies['Genres'] = movies['Genres'].apply(lambda x : x.split('|'))

In [ ]:
movies.head(5)

,MovieID,Title,Genres
0,1,Toy Story (1995),"[Animation, Children's, Comedy]"
1,2,Jumanji (1995),"[Adventure, Children's, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [ ]:
movies['Genres'] =movies['Genres'] .apply(lambda x : " ".join(x))

In [ ]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation Children's Comedy
1,2,Jumanji (1995),Adventure Children's Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
#ratings.drop(['Timestamp'],axis = 1,inplace = True)

In [ ]:
frames = [movies,ratings_temp]
ratings_temp = pd.merge(movies,ratings_temp, on = 'MovieID')

In [ ]:
ratings_temp = pd.merge(ratings_temp, users, on = 'User_ID')

ratings_temp.head()

,MovieID,Title,Genres,User_ID,Rating,Timestamp,Gender,Age,Occupation,Zip-code
0,1,Toy Story (1995),Animation Children's Comedy,6035,4,956712849,F,3,1,78734
1,3,Grumpier Old Men (1995),Comedy Romance,6035,1,956713372,F,3,1,78734
2,4,Waiting to Exhale (1995),Comedy Drama,6035,2,956712120,F,3,1,78734
3,5,Father of the Bride Part II (1995),Comedy,6035,1,956713165,F,3,1,78734
4,7,Sabrina (1995),Comedy Romance,6035,3,956713027,F,3,1,78734


In [ ]:
ratings_temp.sort_values(by = 'Timestamp', inplace = True)
ratings_temp = ratings_temp.reset_index()
ratings_temp.head()

,index,MovieID,Title,Genres,User_ID,Rating,Timestamp,Gender,Age,Occupation,Zip-code
0,11223,858,"Godfather, The (1972)",Action Crime Drama,6040,4,956703932,M,3,6,11106
1,11403,2384,Babe: Pig in the City (1998),Children's Comedy,6040,4,956703954,M,3,6,11106
2,11214,593,"Silence of the Lambs, The (1991)",Drama Thriller,6040,5,956703954,M,3,6,11106
3,11362,1961,Rain Man (1988),Drama,6040,4,956703977,M,3,6,11106
4,11373,2019,Seven Samurai (The Magnificent Seven) (Shichin...,Action Drama,6040,5,956703977,M,3,6,11106


In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
ratings_temp['Rating'] = ratings_temp.Rating.map(lambda x : x -1 )
ratings_temp.sort_values(by = 'Timestamp', inplace = True)
ratings_temp.head()

,index,MovieID,Title,Genres,User_ID,Rating,Timestamp,Gender,Age,Occupation,Zip-code
0,11223,858,"Godfather, The (1972)",Action Crime Drama,6040,3,956703932,M,3,6,11106
1,11403,2384,Babe: Pig in the City (1998),Children's Comedy,6040,3,956703954,M,3,6,11106
2,11214,593,"Silence of the Lambs, The (1991)",Drama Thriller,6040,4,956703954,M,3,6,11106
3,11362,1961,Rain Man (1988),Drama,6040,3,956703977,M,3,6,11106
4,11373,2019,Seven Samurai (The Magnificent Seven) (Shichin...,Action Drama,6040,4,956703977,M,3,6,11106


In [ ]:
df_1=ratings_temp.copy()

In [ ]:
# #For Genres
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus_Genres = []
for i in range(0, len(df_1)):
    review = re.sub('[^a-zA-Z]', ' ', df_1['Genres'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus_Genres.append(review)

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
ratings_temp['text'] = ratings_temp.apply(lambda x : x['Title']+" " + x['Genres'], axis = 1)
ratings_temp.head()

,index,MovieID,Title,Genres,User_ID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,text
0,11223,858,"Godfather, The (1972)",Action Crime Drama,6040,3,956703932,M,3,6,11106,"Godfather, The (1972) Action Crime Drama"
1,11403,2384,Babe: Pig in the City (1998),Children's Comedy,6040,3,956703954,M,3,6,11106,Babe: Pig in the City (1998) Children's Comedy
2,11214,593,"Silence of the Lambs, The (1991)",Drama Thriller,6040,4,956703954,M,3,6,11106,"Silence of the Lambs, The (1991) Drama Thriller"
3,11362,1961,Rain Man (1988),Drama,6040,3,956703977,M,3,6,11106,Rain Man (1988) Drama
4,11373,2019,Seven Samurai (The Magnificent Seven) (Shichin...,Action Drama,6040,4,956703977,M,3,6,11106,Seven Samurai (The Magnificent Seven) (Shichin...


In [ ]:
ratings_temp['text'].apply(lambda x : x.split().__len__()).max()

16

In [ ]:
from tensorflow.keras.layers import TextVectorization
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(ratings_temp['text'].to_list())

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'comedy', 'drama', 'the', 'action', 'thriller',
       'scifi', 'romance', 'adventure', '1999', 'crime', 'of', 'horror',
       'childrens', 'war', '1998', '1997', '1995', '1996'], dtype='<U16')

In [ ]:
x = encoder(ratings_temp['text'].to_list())

In [ ]:
y = tf.one_hot(ratings_temp['Rating'], depth = 5)

In [ ]:
embedding_vector_features=32
model=Sequential()
model.add(Embedding(VOCAB_SIZE,embedding_vector_features,input_length=16))
model.add(LSTM(32))
model.add(Dense(5,activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 16, 32)            32000     
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 5)                 165       
                                                                 
Total params: 40,485
Trainable params: 40,485
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(0.002), loss = 'categorical_crossentropy',metrics = 'accuracy')

In [ ]:
### Finally Training
model.fit(x,y,epochs=32,batch_size=256, validation_split=0.2)

Epoch 1/32
3126/3126 [==============================] - 73s 23ms/step - loss: 1.3590 - accuracy: 0.3871 - val_loss: 1.3484 - val_accuracy: 0.3957
Epoch 2/32
3126/3126 [==============================] - 71s 23ms/step - loss: 1.3427 - accuracy: 0.3952 - val_loss: 1.3410 - val_accuracy: 0.3984
Epoch 3/32
3126/3126 [==============================] - 70s 22ms/step - loss: 1.3377 - accuracy: 0.3973 - val_loss: 1.3383 - val_accuracy: 0.3999
Epoch 4/32
3126/3126 [==============================] - 70s 22ms/step - loss: 1.3345 - accuracy: 0.3988 - val_loss: 1.3377 - val_accuracy: 0.3988
Epoch 5/32
3126/3126 [==============================] - 72s 23ms/step - loss: 1.3317 - accuracy: 0.4001 - val_loss: 1.3343 - val_accuracy: 0.4014
Epoch 6/32
3126/3126 [==============================] - 70s 22ms/step - loss: 1.3295 - accuracy: 0.4010 - val_loss: 1.3346 - val_accuracy: 0.4005
Epoch 7/32
3126/3126 [==============================] - 71s 23ms/step - loss: 1.3280 - accuracy: 0.4014 - val_loss: 1.3298 -

In [ ]:
p = 0
for genre in movies['Genres']:
    words = genre.split('|')
    movies['Genres'][p] = words
    p = p+1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
!pip install listToString

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement listToString (from versions: none)
ERROR: No matching distribution found for listToString


In [ ]:
p = 0
for i in range(0,len(movies)):
    q = listToString(movies['Genres'][p])
    
    movies['Genres'][p] = q
    p = p +1

NameError: ignored

In [ ]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),[Animation Children's Comedy]
1,2,Jumanji (1995),[Adventure Children's Fantasy]
2,3,Grumpier Old Men (1995),[Comedy Romance]
3,4,Waiting to Exhale (1995),[Comedy Drama]
4,5,Father of the Bride Part II (1995),[Comedy]


In [ ]:
df_check = df_1.copy()

In [ ]:
df_check.reset_index(inplace = True)

In [ ]:
#For Gender
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus_G = []
for i in range(0, len(df_check)):
    
    review = df_check['Gender'][i]
    corpus_G.append(review)
    
voc_size = 2
onehot_G=[one_hot(words,voc_size)for words in corpus_G] 

sent_length=30
embedded_gender=pad_sequences(onehot_G,padding='pre',maxlen=sent_length)

In [ ]:
#For Age
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus_age = []
for i in range(0, len(df_check)):
    
    review = df_check['Age'][i]
    corpus_age.append(review)

#For Genres
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus_Genres = []
for i in range(0, len(df_check)):
    
    review = re.sub('[^a-zA-Z]', ' ', df_check['Genres'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus_Genres.append(review)

voc_size = 100
onehot_Genres=[one_hot(words,voc_size)for words in corpus_Genres] 

sent_length=6
embedded_genres=pad_sequences(onehot_Genres,padding='pre',maxlen=sent_length)

# From Movies

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus_Movies = []
for i in range(0, len(df_check)):
    review = re.sub('[^a-zA-Z]', ' ', df_check['Title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus_Movies.append(review)
    
voc_size = 400
onehot_Movies=[one_hot(words,voc_size)for words in corpus_Movies] 

sent_length=30
embedded_Movies=pad_sequences(onehot_Movies,padding='pre',maxlen=sent_length)

In [ ]:
voc_size = 400
onehot_Mt=[one_hot(words,voc_size)for words in corpus_Mt] 

NameError: ignored

In [ ]:
sent_length=6
embedded_Mt=pad_sequences(onehot_Gt,padding='pre',maxlen=sent_length)

NameError: ignored

In [ ]:
X_1t = np.array(embedded_gender)
X_2t = np.array(corpus_age)
X_3t = np.array(embedded_genres)
X_4t = np.array(embedded_Movies)

In [ ]:
X_t = [X_2t,X_3t]

In [ ]:
y_new = model.predict(X_t)
# tf.argmax(y_new, axis = 1)

ValueError: ignored

In [ ]:
y_new

In [ ]:
a = []
for i in range(0, len(y_new)): 
    q = 1*y_new[i][0] + 2*y_new[i][1] + 3*y_new[i][2] + 4*y_new[i][3] + 5*y_new[i][4]
    a.append(q)

In [ ]:
df_12 = pd.DataFrame(a)

In [ ]:
df_12

In [ ]:
df_12['index'] = df_12.index

In [ ]:
df_main = pd.merge(df_12,df_1, on = 'index')
df_main

In [ ]:
movies['i'] = movies.index

In [ ]:
df_12['i'] = df_12.index

In [ ]:
movies

In [ ]:
df_main.sort_values(by = 0, inplace = True)

In [ ]:
df_main[0].describe()

In [ ]:
df_main.rename(columns={0: 'Score'}, inplace=True)

In [ ]:
df_1

In [ ]:
df_main.tail(20)

In [ ]:
df_reco = df_main.copy()
df_reco.drop(['Score'], axis = 1, inplace = True)

In [ ]:
df_main['Score'].value_counts()

In [ ]:
df_main.reset_index(inplace = True)

In [ ]:
df_main.tail(10)

In [ ]:
df_main[df_main['MovieID'] == 48]

In [ ]:
#Testing
df_test = df_1[df_1['User_ID'] == 1]
df_test = df_test[df_test.ratings > 2]

df_reco = df_main.tail(100)
total = df_test.size()



s1 = pd.merge(df_test, df_reco, how='inner', on=['Title'])
present = s1.size()

ratio = present/total

In [ ]:
# 1) User wise - 1,2,3....
# 2) Time Stamp - 
# session based means giving in sequence

In [ ]:
# 1) Session Based logic
# 2) Single User Personalized
# 3) Multiple users are watching

# Personalized Common Recommendation System 